## Imports and Config

In [64]:
import gymnasium as gym
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import random
from collections import namedtuple, deque
import datetime

import matplotlib.pyplot as plt

In [65]:
BUFFER_SIZE = 100000
BATCH_SIZE = 64
GAMMA = 0.99
LR = 0.0005
UPDATE_EVERY = 20

In [66]:
env = gym.make('CartPole-v1')

state_shape = env.observation_space.shape[0]
n_actions = env.action_space.n      

state = env.reset(seed=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [67]:
class MCRNetwork(nn.Module):

    def __init__(
        self,
        state_size: int,
        action_size: int,
        seed: int = 0,
        fc1_units: int = 128,
        fc2_units: int = 64,
    ):
        """Initialize parameters and build model."""
        super(MCRNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.state_size = state_size
        self.action_size = action_size

        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.output_layer = nn.Linear(fc2_units, action_size)

    def forward(self, state: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        output = F.softmax(self.output_layer(x), dim=1)
        return output

    def get_action(self, state):

        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        action_values = self.forward(state)
        chosen_action = np.random.choice(
            np.arange(self.action_size), p=action_values.cpu().data.numpy().flatten()
        )
        log_p = torch.log(action_values.squeeze(0)[chosen_action])
        return chosen_action, log_p

## Agent for Monte-Carlo

In [68]:
class MC_Agent:

    def __init__(self, state_size: int, action_size: int, seed: int, baseline: bool):
        """Agent-Environment Interaction"""
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        """ MC-network """
        self.mc_network = MCRNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.mc_network.parameters(), lr=LR)

        """ Initialize time step (for updating every UPDATE_EVERY steps) """
        self.t_step = 0

        """ Collecing log of probabilities for gradient descent update """
        self.log_probs = []

        """ Storing rewards for updates """
        self.rewards_list = []

        """ Baseline flag, if true, normalize with Value function obtained by TD(0) """
        self.baseline = baseline
        
    def act(self, state: np.ndarray, eps: float = 0.0) -> int:

        chosen_action, log_p = self.mc_network.get_action(state)

        return chosen_action, log_p
        
    def step(self, state: np.ndarray, action: int, reward: float, log_prob: float, next_state: np.ndarray, done: bool):
        
        self.log_probs.append(log_prob)
        self.rewards_list.append(reward)

    ### Implement baseline here, TD(0) updates
        if self.baseline:
            print("implement this")

        if done:
            self.learn(gamma = 0.99)
            return

    def learn(self, gamma: float):

        # Computing G value at each time step
        G_t = [0] * len(self.log_probs) 
        val = 0
        for i in range(len(self.log_probs)-1, -1, -1):
            val *= gamma 
            val += self.rewards_list[i]
            G_t[i] = val

        # From list to a tensor
        G_t = torch.Tensor(G_t)
        # G_t = (G_t - G_t.mean()) / (G_t.std() + 1e-9)

    ### Update this
        if self.baseline:
            G_t -= 0

        gradient = []

        for i in range(len(self.log_probs)):
            gradient.append(-G_t[i] * self.log_probs[i])

        self.optimizer.zero_grad()
        gradient = torch.stack(gradient).sum()
        gradient.backward()
        self.optimizer.step()

        # for param in self.mc_network.parameters():
        #     param.grad.data.clamp_(-1, 1)

In [69]:
class MC_Trainer(MC_Agent):

    def __init__(self, state_size: int, action_size: int, seed: int, baseline: bool):
        super(MC_Trainer, self).__init__(state_size, action_size, seed, baseline)

    def train(self, n_episodes: int = 1000, max_t: int = 1000, eps_start: float = 1.0, eps_end: float = 0.01,eps_decay: float = 0.995):
        scores_window = deque(maxlen=100)
        scores_list = []
        eps = eps_start

        for i_episode in range(1, n_episodes + 1):
            state, _ = env.reset()
            score = 0
            self.log_probs.clear()

            for t in range(max_t):
                action, log_p = self.act(state, eps)
                # print('action is ', action)
                next_state, reward, done, _, _ = env.step(action)
                score += reward
                
                self.step(state, action, reward, log_p, next_state, done)
                state = next_state
                
                if done:
                    break
        
            scores_window.append(score)
            scores_list.append(score)

            eps = max(eps_end, eps_decay * eps)

            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
            
            if i_episode % 100 == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            if np.mean(scores_window) >= 195.0:
                print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
                torch.save(self.mc_network.state_dict(), 'checkpoint_mc.pth')
                break
        
        return scores_list

In [70]:
trainer = MC_Trainer(state_shape, n_actions, seed=0, baseline=False)
scores = trainer.train()            

Episode 100	Average Score: 29.21
Episode 200	Average Score: 84.45
Episode 300	Average Score: 187.95
Episode 307	Average Score: 195.90
Environment solved in 207 episodes!	Average Score: 195.90


In [71]:
scores

[13.0,
 14.0,
 18.0,
 21.0,
 14.0,
 36.0,
 21.0,
 48.0,
 80.0,
 26.0,
 13.0,
 22.0,
 14.0,
 52.0,
 43.0,
 12.0,
 13.0,
 31.0,
 35.0,
 21.0,
 47.0,
 13.0,
 17.0,
 22.0,
 22.0,
 11.0,
 24.0,
 25.0,
 19.0,
 23.0,
 11.0,
 20.0,
 47.0,
 15.0,
 14.0,
 30.0,
 45.0,
 21.0,
 23.0,
 17.0,
 16.0,
 17.0,
 20.0,
 15.0,
 16.0,
 22.0,
 15.0,
 19.0,
 102.0,
 19.0,
 32.0,
 14.0,
 9.0,
 15.0,
 12.0,
 35.0,
 16.0,
 105.0,
 30.0,
 26.0,
 26.0,
 24.0,
 69.0,
 14.0,
 17.0,
 50.0,
 52.0,
 25.0,
 16.0,
 46.0,
 18.0,
 26.0,
 66.0,
 13.0,
 17.0,
 47.0,
 23.0,
 18.0,
 19.0,
 30.0,
 28.0,
 19.0,
 41.0,
 17.0,
 47.0,
 38.0,
 30.0,
 32.0,
 29.0,
 57.0,
 22.0,
 44.0,
 29.0,
 34.0,
 31.0,
 40.0,
 34.0,
 37.0,
 97.0,
 31.0,
 75.0,
 16.0,
 74.0,
 21.0,
 34.0,
 106.0,
 33.0,
 20.0,
 19.0,
 76.0,
 106.0,
 41.0,
 37.0,
 84.0,
 56.0,
 24.0,
 57.0,
 75.0,
 24.0,
 81.0,
 67.0,
 25.0,
 42.0,
 22.0,
 33.0,
 20.0,
 163.0,
 56.0,
 100.0,
 56.0,
 48.0,
 62.0,
 28.0,
 76.0,
 44.0,
 125.0,
 108.0,
 22.0,
 68.0,
 137.0,
 62.0,
 78.0